In [1]:
# !pip install tweepy

In [2]:
import tweepy
import pandas as pd
import json
from pandas.io.json import json_normalize

In [3]:
#### Enter API key and secret ####
consumer_key = '192H10sFeuLbF5hlfO4bJP1Uw'
consumer_key_secret = 'STE7kXLfy3mBf317onMTzBe3t3y7wRiRicrdHKBvQw4Ev5oZeE'
access_token = '700724367404085248-EtoB8C7nnaqEWeEtouJwegKPmwj2FLn'
access_token_secret = 'jhdoWkbpMMhe5Bi1pfUIuxaluNZRgVc1QhYFR0Nb7f7ck'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

### Define Helper Functions

In [5]:
# function to convert _json to JSON
def jsonify_tweepy(tweepy_object):
    json_str = json.dumps(tweepy_object._json)
    return json.loads(json_str)

# flatten nested json
def flatten_json(y):
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(y)
    return out

In [6]:
# df = pd.json_normalize(flatten_json(jsonify_tweepy(api.get_status(str(1221958334661779458)))))
# df.columns

### Read input file

In [7]:
someTweets = pd.read_csv(r'C:\Users\siddh\Desktop\MSA\Acads\CSE 6242\Siddharth.csv')
someTweets.head()

,Unnamed: 0,Unnamed: 0.1,tweet_id,sentiment,emotion
0,45605293,160115920,1381223045600272387,negative,fear
1,45605294,160115932,1381223054357884942,negative,anger
2,45605295,160115934,1381223052109877248,neutral or mixed,no specific emotion
3,45605296,160115935,1381223055876325379,negative,anger
4,45605297,160115940,1381223067062571014,negative,fear


In [8]:
# Specify which rows of dataframe to read
tweet_ids = someTweets['tweet_id'][0:10000]

# Choose columns to extract and initialize dataframe
column_names = ['id', 'text', 'user_location', 'created_at', 'retweet_count', 'favorite_count', 'user_followers_count', 'lang']
df = pd.DataFrame()

In [9]:
len(tweet_ids)

10000

### Create chunks of data to feed into API

In [16]:
n_chunks = 5
size = 10000
start = 0

chunks = [someTweets['tweet_id'][start + (size*i): (start + (i+1)*size)] for i in range(0, n_chunks)]
# chunks

[0       1381223045600272387
 1       1381223054357884942
 2       1381223052109877248
 3       1381223055876325379
 4       1381223067062571014
                ...         
 9995    1381280398446010368
 9996    1381280401562476547
 9997    1381280419128217605
 9998    1381280423439831042
 9999    1381280430637338628
 Name: tweet_id, Length: 10000, dtype: int64,
 10000    1381280438635745284
 10001    1381280450694557698
 10002    1381280458261037056
 10003    1381280454670757890
 10004    1381280460186206208
                 ...         
 19995    1381335773602058243
 19996    1381335777804648451
 19997    1381335788655271936
 19998    1381335802542649344
 19999    1381335802161020928
 Name: tweet_id, Length: 10000, dtype: int64,
 20000    1381335817268891649
 20001    1381335823140880387
 20002    1381335824868970499
 20003    1381335830766182400
 20004    1381335832410341377
                 ...         
 29995    1381397743315398658
 29996    1381397740127682560
 29997    138139776

### Loop over tweets and append to dataframe

In [17]:
t = 0
s = 0
for i, chunk in enumerate(chunks):
    print('=== Started reading tweets from Twitter API for chunk {} ==='.format(i+1))
    df = pd.DataFrame()
    for id, tweet in enumerate(chunk):
        try:
            if (id+1)%100 == 0:
                print('... Finished reading {}/{} tweets ...'.format(id+1, len(chunk)))
            tweet_data = pd.json_normalize(flatten_json(jsonify_tweepy(api.get_status(tweet))))
            t += 1
            df = df.append(tweet_data)
#             df = df.append(tweet_data[column_names])
        except:
            s += 1
            continue
    
    # write output in chunks
    df.to_csv(r'C:\Users\siddh\Desktop\MSA\Acads\CSE 6242\output-'+str(i+1)+'.csv', index=False)

print('\n')
print('***** FINISHED EXTRACTION *****')
print('{} tweets read successfully'.format(t))
print('{} tweets skipped'.format(s))

Rate limit reached. Sleeping for: 351


=== Started reading tweets from Twitter API for chunk 1 ===
... Finished reading 100/10000 tweets ...
... Finished reading 200/10000 tweets ...
... Finished reading 300/10000 tweets ...
... Finished reading 400/10000 tweets ...
... Finished reading 500/10000 tweets ...
... Finished reading 600/10000 tweets ...
... Finished reading 700/10000 tweets ...
... Finished reading 800/10000 tweets ...
... Finished reading 900/10000 tweets ...


Rate limit reached. Sleeping for: 718


... Finished reading 1000/10000 tweets ...
... Finished reading 1100/10000 tweets ...
... Finished reading 1200/10000 tweets ...
... Finished reading 1300/10000 tweets ...
... Finished reading 1400/10000 tweets ...
... Finished reading 1500/10000 tweets ...
... Finished reading 1600/10000 tweets ...
... Finished reading 1700/10000 tweets ...
... Finished reading 1800/10000 tweets ...


Rate limit reached. Sleeping for: 668


...

... Finished reading 3500/10000 tweets ...
... Finished reading 3600/10000 tweets ...
... Finished reading 3700/10000 tweets ...
... Finished reading 3800/10000 tweets ...
... Finished reading 3900/10000 tweets ...
... Finished reading 4000/10000 tweets ...
... Finished reading 4100/10000 tweets ...
... Finished reading 4200/10000 tweets ...
... Finished reading 4300/10000 tweets ...


Rate limit reached. Sleeping for: 398


... Finished reading 4400/10000 tweets ...
... Finished reading 4500/10000 tweets ...
... Finished reading 4600/10000 tweets ...
... Finished reading 4700/10000 tweets ...
... Finished reading 4800/10000 tweets ...
... Finished reading 4900/10000 tweets ...
... Finished reading 5000/10000 tweets ...
... Finished reading 5100/10000 tweets ...
... Finished reading 5200/10000 tweets ...


Rate limit reached. Sleeping for: 271


... Finished reading 5300/10000 tweets ...
... Finished reading 5400/10000 tweets ...
... Finished reading 5500/10000 tweets ...
... Finished reading 5600/10000 tweets ...
... Finished reading 5700/10000 tweets ...
... Finished reading 5800/10000 tweets ...
... Finished reading 5900/10000 tweets ...
... Finished reading 6000/10000 tweets ...
... Finished reading 6100/10000 tweets ...
... Finished reading 6200/10000 tweets ...
... Finished reading 6300/10000 tweets ...
... Finished reading 6400/10000 tweets ...
... Finished reading 6500/10000 tweets ...
... Finished reading 6600/10000 tweets ...
... Finished reading 6700/10000 tweets ...
... Finished reading 6800/10000 tweets ...
... Finished reading 6900/10000 tweets ...
... Finished reading 7000/10000 tweets ...
... Finished reading 7100/10000 tweets ...
... Finished reading 7200/10000 tweets ...
... Finished reading 7300/10000 tweets ...
... Finished reading 7400/10000 tweets ...
... Finished reading 7500/10000 tweets ...
... Finishe

### Concatenate output files

In [18]:
path = r'C:\Users\siddh\Desktop\MSA\Acads\CSE 6242'

df1 = pd.read_csv(path+r'\output-1.csv')
df2 = pd.read_csv(path+r'\output-2.csv')
df3 = pd.read_csv(path+r'\output-3.csv')
df4 = pd.read_csv(path+r'\output-4.csv')
df5 = pd.read_csv(path+r'\output-5.csv')

C:\Users\siddh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (313,314,315,316,317,318,319,320,329,330,331,338,339,340,341,343,344,345,355,356,357,358,359,362,363,364,365,366,367,368,369,372,376,380,381,384,385,386,387,388,389,390,391,392,393,394,395,396,398,399,400,401,402,403,404,405,424,425,426,427,428,429,432,435,438,441,446,447,448,449,450,451,454,457,460,463,468,469,470,471,472,473,476,479,482,485,512,513,514,515,520,523,526,529,532,535,536,545,546,547,548,549,550,553,556,559,562,567,568,569,570,571,572,575,578,581,584,589,590,591,592,593,594,597,600,603,606,607,608,609,613,614,615,618,621,622,627,628,633,634,639,640,645,646,651,656,657,658,659,660,661,662,667,670,673,676,679,682,687,688,689,690,691,694,697,700,703,706,707,708,711,714,727,730,731,732,733,734,735,736,737,740,744,748,749,752,753,754,755,756,757,758,759,760,761,762,763,764,766,767,768,769,770,771,772,773,774,775,780,781,786,789,790,791,794,795,796,801,802,803,806,807,808) ha

In [19]:
df_out = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

df_out.to_csv(path+r'\tweets_full_output.csv', index=False)

In [25]:
# s = 0
# print('=== Started reading tweets from Twitter API ===')
# for id, tweet in enumerate(tweet_ids):
#     try:
#         if (id+1)%100 == 0:
#             print('... Finished reading {}/{} tweets ...'.format(id+1, len(tweet_ids)))
#         tweet_data = pd.json_normalize(flatten_json(jsonify_tweepy(api.get_status(tweet))))
#         df = df.append(tweet_data[column_names])
#     except:
#         s += 1
#         continue
        
# print('\n')
# print('{} tweets read successfully'.format(df.shape[0]))
# print('{} tweets skipped'.format(s))

=== Tweet 13 Skipped ===
=== Tweet 21 Skipped ===
=== Tweet 38 Skipped ===
=== Tweet 64 Skipped ===
=== Tweet 86 Skipped ===
...Finished reading 100/9000 tweets...
=== Tweet 113 Skipped ===
=== Tweet 140 Skipped ===
=== Tweet 143 Skipped ===
=== Tweet 149 Skipped ===
=== Tweet 152 Skipped ===
=== Tweet 163 Skipped ===
=== Tweet 174 Skipped ===
=== Tweet 175 Skipped ===
=== Tweet 180 Skipped ===
...Finished reading 200/9000 tweets...
=== Tweet 205 Skipped ===
=== Tweet 219 Skipped ===
=== Tweet 223 Skipped ===
=== Tweet 239 Skipped ===
=== Tweet 242 Skipped ===
=== Tweet 254 Skipped ===
=== Tweet 255 Skipped ===
...Finished reading 300/9000 tweets...
=== Tweet 319 Skipped ===
=== Tweet 339 Skipped ===
=== Tweet 341 Skipped ===
=== Tweet 343 Skipped ===
=== Tweet 344 Skipped ===
=== Tweet 353 Skipped ===
=== Tweet 359 Skipped ===
=== Tweet 362 Skipped ===
=== Tweet 364 Skipped ===
=== Tweet 373 Skipped ===
=== Tweet 379 Skipped ===
=== Tweet 381 Skipped ===
=== Tweet 387 Skipped ===
...F

Rate limit reached. Sleeping for: 773


=== Tweet 917 Skipped ===
=== Tweet 924 Skipped ===
=== Tweet 928 Skipped ===
=== Tweet 979 Skipped ===
...Finished reading 1000/9000 tweets...
=== Tweet 1015 Skipped ===
=== Tweet 1022 Skipped ===
=== Tweet 1033 Skipped ===
=== Tweet 1039 Skipped ===
=== Tweet 1042 Skipped ===
=== Tweet 1044 Skipped ===
=== Tweet 1059 Skipped ===
=== Tweet 1074 Skipped ===
=== Tweet 1084 Skipped ===
...Finished reading 1100/9000 tweets...
=== Tweet 1104 Skipped ===
=== Tweet 1109 Skipped ===
=== Tweet 1114 Skipped ===
=== Tweet 1145 Skipped ===
=== Tweet 1156 Skipped ===
=== Tweet 1157 Skipped ===
=== Tweet 1180 Skipped ===
=== Tweet 1184 Skipped ===
=== Tweet 1197 Skipped ===
...Finished reading 1200/9000 tweets...
=== Tweet 1226 Skipped ===
=== Tweet 1227 Skipped ===
=== Tweet 1236 Skipped ===
=== Tweet 1252 Skipped ===
=== Tweet 1258 Skipped ===
=== Tweet 1266 Skipped ===
=== Tweet 1273 Skipped ===
=== Tweet 1275 Skipped ===
=== Tweet 1278 Skipped ===
=== Tweet 1288 Skipped ===
...Finished reading 

Rate limit reached. Sleeping for: 746


=== Tweet 1804 Skipped ===
=== Tweet 1810 Skipped ===
=== Tweet 1830 Skipped ===
=== Tweet 1852 Skipped ===
=== Tweet 1882 Skipped ===
=== Tweet 1887 Skipped ===
=== Tweet 1888 Skipped ===
=== Tweet 1898 Skipped ===
=== Tweet 1899 Skipped ===
...Finished reading 1900/9000 tweets...
=== Tweet 1908 Skipped ===
=== Tweet 1916 Skipped ===
=== Tweet 1926 Skipped ===
=== Tweet 1942 Skipped ===
=== Tweet 1952 Skipped ===
=== Tweet 1954 Skipped ===
=== Tweet 1957 Skipped ===
=== Tweet 1958 Skipped ===
=== Tweet 1992 Skipped ===
...Finished reading 2000/9000 tweets...
=== Tweet 2032 Skipped ===
=== Tweet 2033 Skipped ===
=== Tweet 2035 Skipped ===
=== Tweet 2059 Skipped ===
=== Tweet 2088 Skipped ===
...Finished reading 2100/9000 tweets...
=== Tweet 2108 Skipped ===
=== Tweet 2115 Skipped ===
=== Tweet 2122 Skipped ===
...Finished reading 2200/9000 tweets...
=== Tweet 2210 Skipped ===
=== Tweet 2266 Skipped ===
...Finished reading 2300/9000 tweets...
=== Tweet 2309 Skipped ===
=== Tweet 2313 Sk

...

=== Tweet 6666 Skipped ===
=== Tweet 6669 Skipped ===
=== Tweet 6678 Skipped ===
=== Tweet 6679 Skipped ===
=== Tweet 6694 Skipped ===
...Finished reading 6700/9000 tweets...
=== Tweet 6700 Skipped ===
=== Tweet 6710 Skipped ===
=== Tweet 6713 Skipped ===
=== Tweet 6751 Skipped ===
=== Tweet 6761 Skipped ===
=== Tweet 6770 Skipped ===
=== Tweet 6796 Skipped ===
...Finished reading 6800/9000 tweets...
=== Tweet 6853 Skipped ===
=== Tweet 6855 Skipped ===
=== Tweet 6860 Skipped ===
=== Tweet 6871 Skipped ===
=== Tweet 6879 Skipped ===
...Finished reading 6900/9000 tweets...
=== Tweet 6900 Skipped ===
=== Tweet 6908 Skipped ===
=== Tweet 6914 Skipped ===
=== Tweet 6917 Skipped ===
=== Tweet 6923 Skipped ===
=== Tweet 6934 Skipped ===
=== Tweet 6958 Skipped ===
...Finished reading 7000/9000 tweets...
=== Tweet 7019 Skipped ===
=== Tweet 7032 Skipped ===
=== Tweet 7044 Skipped ===
=== Tweet 7063 Skipped ===
=== Tweet 7076 Skipped ===
=== Tweet 7088 Skipped ===
...Finished reading 7100/9000 

Rate limit reached. Sleeping for: 686


=== Tweet 7565 Skipped ===
=== Tweet 7575 Skipped ===
=== Tweet 7593 Skipped ===
...Finished reading 7600/9000 tweets...
=== Tweet 7607 Skipped ===
=== Tweet 7616 Skipped ===
=== Tweet 7618 Skipped ===
=== Tweet 7624 Skipped ===
=== Tweet 7656 Skipped ===
=== Tweet 7657 Skipped ===
=== Tweet 7674 Skipped ===
=== Tweet 7675 Skipped ===
=== Tweet 7681 Skipped ===
=== Tweet 7684 Skipped ===
=== Tweet 7691 Skipped ===
=== Tweet 7693 Skipped ===
...Finished reading 7700/9000 tweets...
=== Tweet 7704 Skipped ===
=== Tweet 7707 Skipped ===
=== Tweet 7708 Skipped ===
=== Tweet 7710 Skipped ===
=== Tweet 7722 Skipped ===
=== Tweet 7725 Skipped ===
=== Tweet 7727 Skipped ===
=== Tweet 7742 Skipped ===
=== Tweet 7752 Skipped ===
=== Tweet 7777 Skipped ===
=== Tweet 7782 Skipped ===
=== Tweet 7791 Skipped ===
=== Tweet 7792 Skipped ===
=== Tweet 7799 Skipped ===
...Finished reading 7800/9000 tweets...
=== Tweet 7802 Skipped ===
=== Tweet 7809 Skipped ===
=== Tweet 7824 Skipped ===
=== Tweet 7852 S

Rate limit reached. Sleeping for: 686


=== Tweet 8466 Skipped ===
=== Tweet 8473 Skipped ===
=== Tweet 8486 Skipped ===
=== Tweet 8490 Skipped ===
...Finished reading 8500/9000 tweets...
=== Tweet 8503 Skipped ===
=== Tweet 8510 Skipped ===
=== Tweet 8584 Skipped ===
=== Tweet 8596 Skipped ===
...Finished reading 8600/9000 tweets...
=== Tweet 8610 Skipped ===
=== Tweet 8644 Skipped ===
=== Tweet 8657 Skipped ===
=== Tweet 8662 Skipped ===
=== Tweet 8669 Skipped ===
=== Tweet 8696 Skipped ===
...Finished reading 8700/9000 tweets...
=== Tweet 8749 Skipped ===
=== Tweet 8758 Skipped ===
=== Tweet 8766 Skipped ===
=== Tweet 8777 Skipped ===
=== Tweet 8779 Skipped ===
=== Tweet 8795 Skipped ===
=== Tweet 8798 Skipped ===
...Finished reading 8800/9000 tweets...
=== Tweet 8801 Skipped ===
=== Tweet 8804 Skipped ===
=== Tweet 8827 Skipped ===
=== Tweet 8855 Skipped ===
=== Tweet 8858 Skipped ===
=== Tweet 8864 Skipped ===
=== Tweet 8869 Skipped ===
=== Tweet 8874 Skipped ===
=== Tweet 8875 Skipped ===
=== Tweet 8879 Skipped ===
===

In [26]:
# df.to_csv(r'C:\Users\siddh\Desktop\MSA\Acads\CSE 6242\out_test-2.csv')